In [1]:
import numpy as np
import pandas as pd
import torch
torch.set_default_tensor_type(torch.FloatTensor) 
import copy
import sys
import os
notebook_dir = os.getcwd()
parent_dir = os.path.dirname(notebook_dir)
# Add the parent directory to the Python path
sys.path.append(parent_dir)

from rct_data_generator import *
from outcome_models import *
from plotting_functions import *
from mcmc_bayes_update import *
from eig_comp_utils import *
from research_exp_utils import *



/Users/lucile/causal_info_gain/pjake/lib/python3.9/site-packages/torch/__init__.py:696: UserWarning: torch.set_default_tensor_type() is deprecated as of PyTorch 2.1, please use torch.set_default_dtype() and torch.set_default_device() as alternatives. (Triggered internally at /Users/runner/work/pytorch/pytorch/pytorch/torch/csrc/tensor/python_tensor.cpp:453.)
  _C._set_default_tensor_type(t)


In [5]:
data, x, t, y = get_data(path='./', dataset='IDHP')
XandT = pd.concat([x,t], axis=1)
print(XandT.head())

         x1        x2        x3        x4        x5        x6  x7  x8  x9  \
0 -0.528957 -0.343685  1.129310  0.161811 -0.316815  1.295216   1   0   1   
1 -1.738109 -1.803210  0.384085  2.245823 -0.629611  1.295216   0   0   0   
2 -0.807992 -0.203082 -0.361140 -0.880195  0.809248 -0.526556   0   0   0   
3  0.390344  0.596982 -1.851590 -0.880195 -0.004020 -0.857787   0   0   0   
4 -1.045929 -0.603114  0.011473  0.161811  0.684130 -0.360940   1   0   0   

   x10  ...  x17  x18  x19  x20  x21  x22  x23  x24  x25    T  
0    0  ...    1    1    1    0    0    0    0    0    0  1.0  
1    1  ...    1    1    1    0    0    0    0    0    0  0.0  
2    1  ...    0    1    1    0    0    0    0    0    0  0.0  
3    0  ...    0    1    1    0    0    0    0    0    0  0.0  
4    0  ...    1    1    1    0    0    0    0    0    0  0.0  

[5 rows x 26 columns]


In [11]:
number_of_candidate_sites = 100

min_sample_size_cand = 150
max_sample_size_cand = 300
host_sample_size = 400 
desired_initial_sample_size = 10**4
XandT = XandT.sample(n=desired_initial_sample_size, replace=True, random_state=42)
added_T_coef = 50 # to increase importance of T

outcome_function = None
std_true_y = 1
power_x = 1
power_x_t = 1
sigma_rand_error = 1
true_beta_great_0_prop = 0.8
host_test_size = 2000

exp_parameters = {'number_of_candidate_sites': number_of_candidate_sites+1, 'min_sample_size_cand': min_sample_size_cand, \
                'max_sample_size_cand': max_sample_size_cand, 'host_sample_size': host_sample_size, 'host_test_size': host_test_size, 'outcome_function': outcome_function, \
                'std_true_y': std_true_y, 'power_x': power_x, 'power_x_t': power_x_t}

causal_param_first_index = power_x*np.shape(XandT)[1]

In [12]:
candidate_sites=generating_random_sites_from(XandT, data, exp_parameters, added_T_coef=1, binary_outcome=False)

 /Users/lucile/causal_info_gain/causal_prospective_merge/rct_data_generator.py:264: RuntimeWarning:overflow encountered in exp


In [15]:
for i, cand in candidate_sites.items():
    if np.sum(cand['T'].values)<30: 
        print('less treated')
    if len(cand['T'].values)-np.sum(cand['T'].values)<30:
        print('less untreated')

In [43]:
#dictionnary of random sites
candidate_sites = generating_random_sites_from(XandT, exp_parameters, added_T_coef=50)

for i, cand in candidate_sites.items():
    candidate_sites[i] = pd.concat([cand, data_with_groundtruth.loc[cand.index, 'Y']], axis=1)

 /Users/lucile/causal_info_gain/causal_prospective_merge/rct_data_generator.py:144: RuntimeWarning:overflow encountered in exp


In [44]:
beta = (np.random.randn(152) > true_beta_great_0_prop)
beta = beta - np.mean(beta)

for i, cand in candidate_sites.items():
    candidate_sites[i]["Y"] = candidate_sites[i].drop(columns=["Y"]) @ beta 
    candidate_sites[i]["Y"] += 1 * np.random.randn(len(candidate_sites[i]["Y"]))
    

In [45]:
host = candidate_sites[0]
candidate_sites = {key: value for key, value in candidate_sites.items() if key != 0}
XandT_host, Y_host = torch.from_numpy(host.drop(columns=["Y"]).values), torch.from_numpy(host["Y"].values)

# Prior parameters for Bayesian update on host
d = np.shape(host)[1]-1
prior_mean = torch.zeros(d)
sigma_prior = 1
beta_0, sigma_0_sq, inv_cov_0 = prior_mean, sigma_rand_error,torch.eye(d)
prior_hyperparameters = {'beta_0': beta_0, 'sigma_0_sq': sigma_0_sq,"inv_cov_0":inv_cov_0}

In [46]:
n_samples_outer_expectation_obs = 400
n_samples_inner_expectation_obs = 800
n_samples_outer_expectation_caus = 400
n_samples_inner_expectation_caus = 800

sampling_parameters = {'n_samples_inner_expectation_obs':n_samples_inner_expectation_obs, 'n_samples_outer_expectation_obs':n_samples_outer_expectation_obs, \
                       'n_samples_inner_expectation_caus':n_samples_inner_expectation_caus, 'n_samples_outer_expectation_caus':n_samples_outer_expectation_caus}

eig_results = {"EIG_obs_from_samples": [], 'EIG_caus_from_samples':[], "EIG_obs_closed_form":[], "EIG_caus_closed_form":[], "EIG_obs_bart":[], "EIG_caus_bart":[]}

In [47]:
print(f" % treated in host: {round(100 * host['T'].mean(),2)}%")

 % treated in host: 48.25%


In [48]:
for _,candidate in candidate_sites.items():
    print(f"For a sample size of {np.shape(candidate)[0]}")
    print(f" % treated in candidate: {round(100 * candidate['T'].mean(),2)}%")

For a sample size of 166
 % treated in candidate: 46.99%
For a sample size of 253
 % treated in candidate: 81.82%
For a sample size of 226
 % treated in candidate: 83.19%
For a sample size of 172
 % treated in candidate: 62.79%
For a sample size of 284
 % treated in candidate: 78.17%
For a sample size of 159
 % treated in candidate: 86.16%
For a sample size of 263
 % treated in candidate: 13.69%
For a sample size of 232
 % treated in candidate: 78.02%
For a sample size of 205
 % treated in candidate: 56.1%
For a sample size of 165
 % treated in candidate: 41.82%


In [49]:
for _, candidate in candidate_sites.items():
    X_cand = torch.from_numpy(candidate.drop(columns=["Y"]).values)
    bayes_reg = BayesianLinearRegression(prior_hyperparameters)
    bayes_reg.set_causal_index(causal_param_first_index)
    post_host_parameters = bayes_reg.fit(XandT_host, Y_host)
    n_samples = n_samples_outer_expectation_obs * (n_samples_inner_expectation_obs + 1)

    eig_results["EIG_obs_closed_form"].append(
            bayes_reg.closed_form_obs_EIG(X_cand)
            )
    eig_results["EIG_caus_closed_form"].append(
            bayes_reg.closed_form_causal_EIG(X_cand)
            )

In [50]:
# eig_results["EIG_obs_from_samples"]=[]
# eig_results["EIG_caus_from_samples"]=[]

# for i, candidate in candidate_sites.items():
#     print("from samples "+str(i))
#     X_cand = torch.from_numpy(candidate.drop(columns=["Y"]).values)
#     bayes_reg = BayesianLinearRegression(prior_hyperparameters)
#     bayes_reg.set_causal_index(causal_param_first_index)
#     post_host_parameters = bayes_reg.fit(XandT_host, Y_host)

#     eig_results["EIG_obs_from_samples"].append(
#             bayes_reg.samples_obs_EIG(
#                 X_cand, n_samples_outer_expectation_obs, n_samples_inner_expectation_obs
#             )
#         )
#     eig_results["EIG_caus_from_samples"].append(
#             bayes_reg.samples_causal_EIG(
#                 X_cand, n_samples_outer_expectation_obs, n_samples_inner_expectation_obs
#             )
#         )

In [51]:
# now merge and compute some CATE error
merged_datasets = {}

for i, candidate in candidate_sites.items():
    merged_datasets[i]= pd.concat([host, candidate], axis=0)

In [52]:
cate_diff = {}
merged_mse = []
XandT_host=host.drop(columns=["Y"])

X_zero = XandT_host.copy() # we predict on host with T=0 and T=1
X_zero.iloc[:,causal_param_first_index:] = 0

X_one = XandT_host.copy()
X_one.iloc[:,causal_param_first_index:] = XandT_host.iloc[:,:causal_param_first_index]

### Merging and computing ground truth

In [53]:
from sklearn.metrics import mean_squared_error
from sklearn.linear_model import Ridge
from sklearn.metrics import log_loss

merged_mse = []

for i, candidate in merged_datasets.items():

    XandT_merged = candidate.drop(columns=["Y"])
    Y_merged = candidate['Y']

    learner = Ridge(fit_intercept=True)
    learner.fit(y=Y_merged, X=XandT_merged) # we fit on merged datasets

    true_cate = (X_one - X_zero) @ beta

    pred_cate = learner.predict(X_one)-learner.predict(X_zero)

    merged_mse.append(mean_squared_error(true_cate, pred_cate))


### Comparing our EIGs with ground truth

In [54]:
obs_eig_ranking_closed_form = sorted(range(len(eig_results["EIG_obs_closed_form"])), key=lambda i: eig_results["EIG_obs_closed_form"][i], reverse=True)
print(obs_eig_ranking_closed_form)

caus_eig_ranking_closed_form = sorted(range(len(eig_results["EIG_caus_closed_form"])), key=lambda i: eig_results["EIG_caus_closed_form"][i], reverse=True)
print(caus_eig_ranking_closed_form)

# obs_eig_ranking_from_samples = sorted(range(len(eig_results["EIG_obs_from_samples"])), key=lambda i: eig_results["EIG_obs_from_samples"][i], reverse=True)
# print(obs_eig_ranking_from_samples)

# caus_eig_ranking_from_samples = sorted(range(len(eig_results["EIG_caus_from_samples"])), key=lambda i: eig_results["EIG_caus_from_samples"][i], reverse=True)
# print(caus_eig_ranking_from_samples)

true_cate_ranking = sorted(range(len(merged_mse)), key=lambda i: merged_mse[i], reverse=False) # reverse is False because its error terms
print(true_cate_ranking)

[6, 4, 7, 1, 8, 2, 0, 3, 9, 5]
[7, 4, 8, 6, 1, 2, 3, 0, 9, 5]
[2, 7, 9, 4, 8, 1, 3, 0, 5, 6]


In [63]:
k =[1,3,5]
top_n = None

In [64]:
correlation_with_true_rankings={}

for val in k:
    correlation_with_true_rankings['precision_at_'+str(val)] = []
compare_to_ground_truth(correlation_with_true_rankings, true_cate_ranking, obs_eig_ranking_closed_form, merged_mse=merged_mse, top_n = top_n, k = k)
compare_to_ground_truth(correlation_with_true_rankings, true_cate_ranking, caus_eig_ranking_closed_form,merged_mse=merged_mse, top_n = top_n, k = k)

# compare_to_ground_truth(correlation_with_true_rankings, true_cate_ranking, obs_eig_ranking_from_samples, top_n = top_n, k = k)
# compare_to_ground_truth(correlation_with_true_rankings, true_cate_ranking, caus_eig_ranking_from_samples, top_n = top_n, k = k)


{'precision_at_1': [0.0, 0.0],
 'precision_at_3': [0.3333333333333333, 0.3333333333333333],
 'precision_at_5': [0.6, 0.6],
 'tau': [0.06666666666666667, 0.3333333333333333],
 'rho': [0.06666666666666665, 0.34545454545454546]}

### Baselines

In [65]:
import copy
from sklearn.linear_model import LogisticRegression

### random ranking
random_ranking = np.random.choice(np.arange(1, number_of_candidate_sites+1), size=number_of_candidate_sites, replace=False)


### ranking by sample size
sample_size_order = sorted(candidate_sites.keys(), key=lambda key: -candidate_sites[key].shape[0])


### ranking by similarity of covariate distribution
mean_vector_host = XandT_host.iloc[:,:causal_param_first_index].mean()
cov_matrix_host = XandT_host.iloc[:,:causal_param_first_index].cov()
mvn = multivariate_normal(mean=mean_vector_host, cov=cov_matrix_host, allow_singular=1)
# get log likelihood of candidate sites
log_likelihood_list=[]
for i, candidate in candidate_sites.items():
    log_likelihoods=mvn.logpdf(candidate.iloc[:,:causal_param_first_index].values)
    log_likelihood_list.append(np.mean(log_likelihoods))

similarity_cov_distrib_ranking= sorted(range(len(log_likelihood_list)), key=lambda i: log_likelihood_list[i], reverse=True)

### ranking by similarity of propensity scores
# we fit a propensity score model at target site and store logloss
# for each site: we fit the model further on the cand site and compute log
# nd assess the loss. Sites associated with loss values with higher discrepancy from the host should have distinct 
#treatment allocation scheme, and thus be a better fit. 

ps_model = LogisticRegression(fit_intercept=True)
ps_model.fit(XandT_host.iloc[:,:causal_param_first_index], XandT_host['T'])
t_host_pred = ps_model.predict(XandT_host.iloc[:,:causal_param_first_index])
mse_host = mean_squared_error(t_host_pred, XandT_host['T'])
mse_diff_list = []


for i, candidate in candidate_sites.items():
    # ps_model_copy= copy.deepcopy(ps_model)
    # ps_model_copy.fit(candidate.iloc[:,:causal_param_first_index], candidate['T'])
    t_cand_pred = ps_model.predict(candidate.iloc[:,:causal_param_first_index]) # predict on host!
    mse_cand = abs(mean_squared_error(t_cand_pred, candidate['T']) - mse_host)
    mse_diff_list.append(mse_cand)

similarity_pscore_ranking = sorted(range(len(mse_diff_list)), key=lambda i: mse_diff_list[i], reverse=True) 
# the more diff in pscore the better so reverse=True


print(random_ranking)
print(sample_size_order)
print(similarity_cov_distrib_ranking)
print(similarity_pscore_ranking)

[10  7  1  5  2  4  8  3  9  6]
[5, 7, 2, 8, 3, 9, 4, 1, 10, 6]
[2, 0, 1, 3, 4, 5, 6, 7, 8, 9]
[5, 2, 4, 7, 1, 3, 8, 0, 9, 6]


 /Users/lucile/causal_info_gain/pjake/lib/python3.9/site-packages/sklearn/linear_model/_logistic.py:444: ConvergenceWarning:lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression


In [66]:
list(sample_size_order).index(4)

6

In [67]:
compare_to_ground_truth(correlation_with_true_rankings, true_cate_ranking, list(random_ranking),merged_mse=merged_mse, top_n = top_n, k = k)
compare_to_ground_truth(correlation_with_true_rankings, true_cate_ranking, list(sample_size_order),merged_mse=merged_mse, top_n = top_n, k = k)
compare_to_ground_truth(correlation_with_true_rankings, true_cate_ranking, list(similarity_cov_distrib_ranking),merged_mse=merged_mse, top_n = top_n, k = k)
compare_to_ground_truth(correlation_with_true_rankings, true_cate_ranking, list(similarity_pscore_ranking),merged_mse=merged_mse, top_n = top_n, k = k)

{'precision_at_1': [0.0, 0.0, 0.0, 0.0, 1.0, 0.0],
 'precision_at_3': [0.3333333333333333,
  0.3333333333333333,
  0.3333333333333333,
  0.6666666666666666,
  0.3333333333333333,
  0.3333333333333333],
 'precision_at_5': [0.6, 0.6, 0.4, 0.6, 0.4, 0.6],
 'tau': [0.06666666666666667,
  0.3333333333333333,
  -0.1111111111111111,
  0.15555555555555553,
  -0.022222222222222223,
  0.28888888888888886],
 'rho': [0.06666666666666665,
  0.34545454545454546,
  -0.12727272727272726,
  0.1515151515151515,
  -0.05454545454545454,
  0.32121212121212117]}

### Show results

In [68]:
correlation_with_true_rankings= pd.DataFrame.from_dict(correlation_with_true_rankings)
correlation_with_true_rankings.index = ['obs_closed_form', 'caus_closed_form', 'random', 'sample size', 'similarity_cov_distrib_ranking', 'similarity_pscore_ranking size'] #, 'obs_from_samples', 'caus_from_samples']
correlation_with_true_rankings

,precision_at_1,precision_at_3,precision_at_5,tau,rho
obs_closed_form,0.0,0.333333,0.6,0.066667,0.066667
caus_closed_form,0.0,0.333333,0.6,0.333333,0.345455
random,0.0,0.333333,0.4,-0.111111,-0.127273
sample size,0.0,0.666667,0.6,0.155556,0.151515
similarity_cov_distrib_ranking,1.0,0.333333,0.4,-0.022222,-0.054545
similarity_pscore_ranking size,0.0,0.333333,0.6,0.288889,0.321212


In [69]:
kendalltau(true_cate_ranking, merged_mse)

SignificanceResult(statistic=-0.4222222222222222, pvalue=0.10831349206349207)

## bart stuff

In [70]:
# X_host, T_host, Y_host = host.drop(columns=['T','Y']).values, host['T'].values.astype(np.int32), host['Y'].values

# prior_hyperparameters = {'sigma_0_sq':1, 'p_categorical_pr':0, 'p_categorical_trt':0 }
# predictive_model_parameters={"num_trees_pr":200,"num_trees_trt":100}
# conditional_model_param={"num_trees_pr":200}


# for i, candidate in candidate_sites.items():

#     print("from samples "+str(i))
#     X_cand, T_cand = candidate.drop(columns=['Y','T']).values, candidate['T'].values.astype(np.int32)

#     bcf = BayesianCausalForest(
#         prior_hyperparameters,
#         predictive_model_parameters=predictive_model_parameters,
#         conditional_model_param=conditional_model_param)
#     bcf.store_train_data(X=X_host, T=T_host, Y=Y_host)
    
#     joint_eig = bcf.joint_EIG_calc(X_cand, T_cand, sampling_parameters)

#     results["EIG_obs_bart"].append(joint_eig["Obs EIG"])
#     results["EIG_caus_bart"].append(joint_eig["Causal EIG"])